In [1]:
from pyspark.sql import SparkSession
import sqlite3
from datetime import datetime
import pandas as pd

In [2]:
def read_sqlite_data(db_path, query):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(query)
    data = cursor.fetchall()

    conn.close()
    return data

In [3]:
def execute_query_spark(spark, query):
  try:
    df = spark.sql(query)
  except Exception as e:
    raise Exception(f"Error executing query: {e}")
  return df

In [4]:
db_path = "./mydatabase.db"
driver_path = "./sqlite-jdbc-3.45.3.0.jar"

# Create a SparkSession with connection properties for SQLite
spark = SparkSession.builder \
    .appName("SparkSQLite") \
    .config("spark.sql.jdbc.url", f"jdbc:sqlite://{db_path}") \
    .config("spark.sql.jdbc.driver", "org.sqlite.JDBC") \
    .config("spark.driver.classpath", f"{driver_path}") \
    .getOrCreate()


24/04/25 14:49:14 WARN Utils: Your hostname, strun resolves to a loopback address: 127.0.1.1; using 192.168.0.4 instead (on interface wlp5s0)
24/04/25 14:49:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/25 14:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = read_sqlite_data(db_path,  f"SELECT * FROM {'Historja_pomiarow'}")
pd.DataFrame(df)

,0,1,2,3
0,Czerniawa,DsCzerStraza-NO-1g,2023-11-27 12:00:00,1.93112
1,Czerniawa,DsCzerStraza-NO-1g,2023-11-27 13:00:00,2.17532
2,Czerniawa,DsCzerStraza-NO-1g,2023-11-27 14:00:00,2.18571
3,Czerniawa,DsCzerStraza-NO-1g,2023-11-27 15:00:00,1.49813
4,Czerniawa,DsCzerStraza-NO-1g,2023-11-27 16:00:00,0.98478
...,...,...,...,...
248093,"Bydgoszcz, ul. Warszawska",KpBydWarszaw-NO2-1g,2024-04-25 08:00:00,50.80000
248094,"Bydgoszcz, ul. Warszawska",KpBydWarszaw-NO2-1g,2024-04-25 09:00:00,36.50000
248095,"Bydgoszcz, ul. Warszawska",KpBydWarszaw-NO2-1g,2024-04-25 10:00:00,33.70000
248096,"Bydgoszcz, ul. Warszawska",KpBydWarszaw-NO2-1g,2024-04-25 11:00:00,25.70000


In [6]:
pd.DataFrame(df).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248098 entries, 0 to 248097
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       248098 non-null  object 
 1   1       248098 non-null  object 
 2   2       248098 non-null  object 
 3   3       248098 non-null  float64
dtypes: float64(1), object(3)
memory usage: 7.6+ MB


In [7]:
df = read_sqlite_data(db_path,  f"SELECT * FROM {'Stanowiska_pomiarowe'}")
pd.DataFrame(df)

,0,1,2,3,4,5
0,49,11,tlenek azotu,NO,NO,16
1,50,11,dwutlenek azotu,NO2,NO2,6
2,51,11,tlenki azotu,NOx,NOx,7
3,52,11,ozon,O3,O3,5
4,54,11,pył zawieszony PM10,PM10,PM10,3
...,...,...,...,...,...,...
1891,29676,17880,ozon,O3,O3,5
1892,29677,17880,tlenek węgla,CO,CO,8
1893,29679,17880,pył zawieszony PM10,PM10,PM10,3
1894,29680,17880,dwutlenek azotu,NO2,NO2,6


In [8]:
query_1 = """select Stacje.wojewodztwo,
    avg(wartosc),
    max(wartosc),
    min(wartosc)
    from Historja_pomiarow  
    join Stanowiska_pomiarowe on Stanowiska_pomiarowe.wskaznik = Historja_pomiarow.Nazwa_stacji
    join Stacje on Stacje.id = Stanowiska_pomiarowe.id_stacji
    group by Stacje.wojewodztwo"""
analiza_1 = read_sqlite_data(db_path,  query_1)
pd.DataFrame(analiza_1)

""


In [9]:
query_2 = """select Nazwa_stacji, Kod_stanowiska, date(date_time), 
    avg(wartosc),
    max(wartosc),
    min(wartosc)
    from Historja_pomiarow 
    group by Nazwa_stacji, Kod_stanowiska, date(date_time)"""
analiza_2 = read_sqlite_data(db_path,  query_2)
pd.DataFrame(analiza_2)

,0,1,2,3,4,5
0,"Bydgoszcz, pl. Poznański",KpBydPlPozna-C6H6-1g,2023-11-27,5.650000,10.0,1.7
1,"Bydgoszcz, pl. Poznański",KpBydPlPozna-C6H6-1g,2023-11-28,4.408333,10.0,1.5
2,"Bydgoszcz, pl. Poznański",KpBydPlPozna-C6H6-1g,2023-11-29,2.537500,6.1,1.5
3,"Bydgoszcz, pl. Poznański",KpBydPlPozna-C6H6-1g,2023-11-30,2.045833,5.0,1.0
4,"Bydgoszcz, pl. Poznański",KpBydPlPozna-C6H6-1g,2023-12-01,3.808333,5.9,1.9
...,...,...,...,...,...,...
15754,"Zgorzelec, ul. Bohaterów Getta",DsZgorBohGet-PM2.5-24g,2024-03-27,9.500000,9.5,9.5
15755,"Zgorzelec, ul. Bohaterów Getta",DsZgorBohGet-PM2.5-24g,2024-03-28,6.000000,6.0,6.0
15756,"Zgorzelec, ul. Bohaterów Getta",DsZgorBohGet-PM2.5-24g,2024-03-29,6.400000,6.4,6.4
15757,"Zgorzelec, ul. Bohaterów Getta",DsZgorBohGet-PM2.5-24g,2024-03-30,34.100000,34.1,34.1
